In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

So the code below should import a csv that has already been converted completely to numerical data. The commented out code was used to format the data properly.

In [12]:
# df = pd.read_csv('numerical_df.csv')
df = pd.read_csv('/content/Copy of VolleyballBlockingData - MASTERv3.csv')

This code will turn all the NaN values to -1, and then go through and bin each column so that I can run the CategoricalNB on it. Note that features with fewer than 5 unique values are dropped so that all the values can be binned. The data is then split into x and y DataFrames, with y being the target column.

In [28]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
import warnings
warnings.filterwarnings("ignore")

def convert_to_numerical(df):
  label_encoder = LabelEncoder()
  for col in df.columns:
    df[col] = label_encoder.fit_transform(df[col])
  df.fillna(-1, inplace=True)
  return df

df = convert_to_numerical(df)
x = df.drop('Classification_target', axis=1)
y = df['Classification_target']



def binner(x, y):
    for col in x.columns:
        if x[col].nunique() < 5:
            x.drop(col, axis=1, inplace=True)

    def should_bin(column):
        return not all(value == 0 for value in column) and len(column.unique()) >= 5

    for col in x.columns:
        if should_bin(x[col]):
            encoder = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
            x[col] = encoder.fit_transform(x[col].values.reshape(-1, 1)).flatten()

    encoder_y = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
    y = encoder_y.fit_transform(y.values.reshape(-1, 1)).flatten()

    return x, y


Here the model is ran 5 times and all training and test accuracies are reported, as well as the average for each.

In [29]:
x, y = binner(x, y)
train_accuracies = []
test_accuracies = []
for _ in range(5):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)
  model = CategoricalNB()
  model.fit(x_train, y_train)
  y_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)
  train_acc = accuracy_score(y_train, y_train_pred)
  test_acc = accuracy_score(y_test, y_test_pred)
  train_accuracies.append(train_acc)
  test_accuracies.append(test_acc)

train_avg = np.mean(train_accuracies)
test_avg = np.mean(test_accuracies)
train_accuracies.append(train_avg)
test_accuracies.append(test_avg)
results = pd.DataFrame({"Training Accuracy": train_accuracies, "Test Accuracy":test_accuracies})
results = results.rename({len(results)-1: 'Average'})
print(results)

         Training Accuracy  Test Accuracy
0                 0.545045       0.410714
1                 0.576577       0.321429
2                 0.522523       0.446429
3                 0.513514       0.410714
4                 0.509009       0.321429
Average           0.533333       0.382143
